In [1]:
from datetime import datetime
import ee
import geemap
import geetools
import pandas as pd

In [2]:
# Function to add 'date' property (just the day, ignoring hours)
def add_date(image):
    
    date = ee.Date(image.get('model_initialization_datetime')).format('YYYY-MM-dd');
    year = ee.Date(image.get('model_initialization_datetime')).get('year')
    month = ee.Date(image.get('model_initialization_datetime')).get('month')
    week = ee.Date(image.get('model_initialization_datetime')).get('week')
    day = ee.Date(image.get('model_initialization_datetime')).get('day')
    
    forecast_hour = image.get('model_forecast_hour');
    
    
    return image.set({
        "date": date,
        "year": year,
        "month": month,
        "week": week,
        "day": day
    })

In [3]:
def recent_daily_means(date):
    
    filtered = cams_filtered.filter(ee.Filter.eq('date', date))
    mean = filtered.mean()
    
    return mean.set('date', date)

In [4]:
ee.Initialize()

In [5]:
# The images from the time which we will analyze
start_date = '2024-09-01'
end_date = datetime.today().strftime('%Y-%m-%d')

cams_filtered = (ee.ImageCollection('ECMWF/CAMS/NRT')
                   .filterDate(start_date, end_date)
                   .filter(ee.Filter.eq('model_initialization_hour', 0))
                   .filter(ee.Filter.lte('model_forecast_hour', 21))
                   .select('particulate_matter_d_less_than_25_um_surface'))

In [6]:
end_date

'2024-09-12'

In [7]:
cams_filtered = cams_filtered.map(add_date)

In [8]:
# Compute the daily means for the interval of interest
unique_dates = cams_filtered.aggregate_array('date').distinct()
recent_daily_means = ee.ImageCollection(unique_dates.map(recent_daily_means))

In [9]:
# Load the cities feature collection and filter for South American countries
southAmericaISO = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'];
cities = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_outlines")\
              .filter(ee.Filter.inList('CTR_MN_ISO', southAmericaISO))\
              .filter(ee.Filter.gte('P15', 7e4));


In [10]:
# Reduces of the recent images over the cities, getting an average of the territory
def daily_mean_for_cities(image):
    
    # Use reduceRegions to calculate the mean PM2.5 over each city
    
    means = image.reduceRegions(
        collection=cities,                  # The cities feature collection
        reducer=ee.Reducer.mean(),          # Use mean reducer
        scale=44528,                        # Adjust scale based on the dataset resolution
    )
    
    # Add the 'date' property from the image to each feature (city)
    means_with_date = means.map(lambda feature: feature.set('date', image.get('date')))
    return means_with_date

# Apply the mean_for_cities function to all images in the dailyMeans ImageCollection
city_recent_daily_means = recent_daily_means.map(daily_mean_for_cities).flatten()

In [11]:
# Access it as a CSV and save
city_recent_daily_means = geemap.ee_to_df(city_recent_daily_means)

In [14]:
# Saves it as CSV
city_recent_daily_means.to_csv("../output/city-recent-daily-means.csv", index=False)